In [0]:
import os, uuid
import time, datetime
import multiprocessing
from multiprocessing import cpu_count
import pandas as pd
# from azure.cosmos import CosmosClient

In [0]:
container_name="espdata"
storage_account_name = "gw2data2lake2storage"
storage_account_access_key = "8iT17dy9BxdNbEpT68hjdV72+doHN7cGICXAvMF5iI67Jw0gzVYw5jvXE3IYZTD0khpNXKj6lb7vUS9xBzPhpQ=="

connect_str = "DefaultEndpointsProtocol=https;AccountName=gw2data2lake2storage;AccountKey=8iT17dy9BxdNbEpT68hjdV72+doHN7cGICXAvMF5iI67Jw0gzVYw5jvXE3IYZTD0khpNXKj6lb7vUS9xBzPhpQ==;EndpointSuffix=core.windows.net"

file_location = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"
# 写入ETL的时间
filename = './CameraStatus/ETL_date.txt'
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [0]:
def gen_dates(b_date, days):
    day = datetime.timedelta(days=1)
    for i in range(days):
        yield b_date + day*i


def get_date_list(start=None, end=None):
    """
    获取日期列表
    :param start: 开始日期
    :param end: 结束日期
    :return:
    """
    if start is None:
        start = datetime.datetime.strptime("2020/12/01", "%Y/%m/%d")
    if end is None:
        end = datetime.datetime.now()
    data = []
    for d in gen_dates(start, (end-start).days):
        data.append(d)
    data_new = []
    for i in data:
      new = str(i)[:10]
      new_time = new.split('-')
      year , month , day = int(new_time[0]) , int(new_time[1]) , int(new_time[2]) 
      date = str(year) + '/' + str(month) + '/' +str(day) 
      data_new.append(date)
    return data_new

data = get_date_list()
with open(filename,'r') as f: 
  ETL_date = f.read()
time = data[data.index(ETL_date)+1]
time

Out[53]: '2021/3/27'

In [0]:
spark.catalog.setCurrentDatabase("camera")

In [0]:
year , month , day = time.split('/')
year , month , day = int(year) , int(month) , int(day)
cameras = dbutils.fs.ls(file_location)
data = [] # 'camera_name' , 'year' , 'month' , 'day' , 'up_load_image_num','positive','negative'
for camera in cameras:
  camera_name = camera[1]
  img_location = file_location + camera_name  + time # wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000000000T/
  camera_name = camera_name.replace('/','')
  try:
    img = dbutils.fs.ls(img_location)
    # read positive & negtive
    df = spark.read.format("json").load(img_location)
    positive , negative = df[df['score'] >= 128] , df[df['score'] < 128]
    sql = "insert into camerastatus values ('%s',%d,%d,%d,%d,%d,%d)" % (camera_name,year,month,day,len(img),int(positive.count()),int(negative.count()))
    spark.sql(sql)
    print(sql)
  except:
    print(img_location +" does not exist")
    sql = "insert into camerastatus values ('%s',%d,%d,%d,%d,%d,%d)" % (camera_name,year,month,day,0,0,0)
    spark.sql(sql)
    print(sql)
with open(filename,'w') as f:
  f.write(time)
print(time)

wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000000000T/2021/3/27 does not exist
insert into camerastatus values ('00000000000000T',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000000003/2021/3/27 does not exist
insert into camerastatus values ('00000000000003',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/0000000000000t/2021/3/27 does not exist
insert into camerastatus values ('0000000000000t',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000009478/2021/3/27 does not exist
insert into camerastatus values ('00000000009478',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/0000000000947C/2021/3/27 does not exist
insert into camerastatus values ('0000000000947C',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000009A79/2021/3/27 does not exist
insert into camerastatus values ('00000000009A79',2021,3,27,0,0,0)
insert into camerastatus values ('0000000000EB9B',2021,3,27,8623,130,8493)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/0000000000TEST/2021/3/27 does not exist
insert into camerastatus values ('0000000000TEST',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60200924000002/2021/3/27 does not exist
insert into camerastatus values ('60200924000002',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60200925000023/2021/3/27 does not exist
insert into camerastatus values ('60200925000023',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60201121000004/2021/3/27 does not exist
insert into camerastatus values ('60201121000004',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60201121000017/2021/3/27 does not exist
insert into camerastatus values ('60201121000017',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60201122000001/2021/3/27 does not exist
insert into camerastatus values ('60201122000001',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60210117000118/2021/3/27 does not exist
insert into camerastatus values ('60210117000118',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/60210122000057/2021/3/27 does not exist
insert into camerastatus values ('60210122000057',2021,3,27,0,0,0)
wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/LDCSH029/2021/3/27 does not exist
insert into camerastatus values ('LDCSH029',2021,3,27,0,0,0)
2021/3/27

In [0]:
%sql
select * from camerastatus

camera_name,year,month,day,up_load_image_num,positive,negative
00000000000000T,2021,1,1,0,0,0
00000000000000T,2021,1,4,0,0,0
00000000000000T,2021,1,3,0,0,0
00000000000000T,2021,1,2,0,0,0
0000000000947C,2021,1,1,0,0,0
60201121000004,2021,1,3,0,0,0
0000000000EB9B,2021,1,4,0,0,0
00000000009A79,2021,1,3,0,0,0
60201122000001,2021,1,1,0,0,0
00000000009478,2021,1,3,0,0,0


In [0]:
# spark_df.write.format("parquet").mode('append').save("cameraStatus_yesterday")

In [0]:
# def read(): 
#   data = spark.read.format("json").load("wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/0000000000EB9B/2021/4/6/")
#   data = data.toPandas()
#   return data
# for i in range(1):
# #   image_path = location_path +"/" + i[1]
#   p.apply_async(read())
#   display(data)